In [1]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/itv009200/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [4]:
spark.conf.set("spark.sql.unacceptable_rated_pts", 0)
spark.conf.set("spark.sql.very_bad_rated_pts", 100)
spark.conf.set("spark.sql.bad_rated_pts", 250)
spark.conf.set("spark.sql.good_rated_pts", 500)
spark.conf.set("spark.sql.very_good_rated_pts", 650)
spark.conf.set("spark.sql.excellent_rated_pts", 800)

In [5]:
spark.conf.set("spark.sql.unacceptable_grade_pts", 750)
spark.conf.set("spark.sql.very_bad_grade_pts", 1000)
spark.conf.set("spark.sql.bad_grade_pts", 1500)
spark.conf.set("spark.sql.good_grade_pts", 2000)
spark.conf.set("spark.sql.very_good_grade_pts", 2500)

In [6]:
bad_customer_data_final_df = spark.read \
.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.load("/public/trendytech/lendingclubproject/bad/bad_customer_data_final")

In [7]:
bad_customer_data_final_df.createOrReplaceTempView("bad_data_customer")

In [8]:
spark.sql("select * from bad_data_customer limit 10").show()

+--------------------+
|           member_id|
+--------------------+
|514da6011e2f772fb...|
|d3125d00da326895a...|
|4e9cc2faea777e410...|
|8d42ec724248a79d2...|
|c14309ba2311e430d...|
|d9ce4046daf599732...|
|e0ecccbcb94183be6...|
|fecada6499caf7a4d...|
|62d93eb316a3eacf4...|
|e757d51dddd8fa34e...|
+--------------------+



In [10]:
ph_df = spark.sql("select c.member_id, \
   case \
   when p.last_payment_amount < (c.monthly_installment * 0.5) then ${spark.sql.very_bad_rated_pts} \
   when p.last_payment_amount >= (c.monthly_installment * 0.5) and p.last_payment_amount < c.monthly_installment then ${spark.sql.very_bad_rated_pts} \
   when (p.last_payment_amount = (c.monthly_installment)) then ${spark.sql.good_rated_pts} \
   when p.last_payment_amount > (c.monthly_installment) and p.last_payment_amount <= (c.monthly_installment * 1.50) then ${spark.sql.very_good_rated_pts} \
   when p.last_payment_amount > (c.monthly_installment * 1.50) then ${spark.sql.excellent_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   end as last_payment_pts, \
   case \
   when p.total_payment_received >= (c.funded_amount * 0.50) then ${spark.sql.very_good_rated_pts} \
   when p.total_payment_received < (c.funded_amount * 0.50) and p.total_payment_received > 0 then ${spark.sql.good_rated_pts} \
   when p.total_payment_received = 0 or (p.total_payment_received) is null then ${spark.sql.unacceptable_rated_pts} \
   end as total_payment_pts \
from itv009200_lending_club.loans_repayments_new p \
inner join itv009200_lending_club_new.loans1 c on c.loan_id = p.loan_id where member_id NOT IN (select member_id from bad_data_customer)")

In [11]:
ph_df.createOrReplaceTempView("ph_pts")

In [12]:
spark.sql("select * from ph_pts")

member_id,last_payment_pts,total_payment_pts
dcec9334e70f1cc95...,800,650
fc58ca61f51f9dcac...,500,650
2fb62a6ca51063b11...,500,650
488268a5531951622...,800,650
ade6026208e48f5f9...,500,650
7c8b0ca6acddfaeb1...,800,650
a707b7fe7c38bad65...,800,650
1df639cddea30c288...,800,650
22d67005e12d8726d...,500,650
009cf312bd46551b4...,500,650


In [16]:
ldh_ph_df = spark.sql(
    "select p.*, \
        CASE \
        WHEN d.delinq_2yrs = 0 THEN ${spark.sql.excellent_rated_pts} \
        WHEN d.delinq_2yrs BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
        WHEN d.delinq_2yrs BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
        WHEN d.delinq_2yrs > 5 OR d.delinq_2yrs IS NULL THEN ${spark.sql.unacceptable_grade_pts} \
        END AS delinq_pts, \
        CASE \
        WHEN l.pub_rec = 0 THEN ${spark.sql.excellent_rated_pts} \
        WHEN l.pub_rec BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
        WHEN l.pub_rec BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
        WHEN l.pub_rec > 5 OR l.pub_rec IS NULL THEN ${spark.sql.very_bad_rated_pts} \
        END AS public_records_pts, \
        CASE \
        WHEN l.pub_rec_bankruptcies = 0 THEN ${spark.sql.excellent_rated_pts} \
        WHEN l.pub_rec_bankruptcies BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
        WHEN l.pub_rec_bankruptcies BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
        WHEN l.pub_rec_bankruptcies > 5 OR l.pub_rec_bankruptcies IS NULL THEN ${spark.sql.very_bad_rated_pts} \
        END AS public_bankruptcies_pts, \
        CASE \
        WHEN l.inq_last_6mths = 0 THEN ${spark.sql.excellent_rated_pts} \
        WHEN l.inq_last_6mths BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
        WHEN l.inq_last_6mths BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
        WHEN l.inq_last_6mths > 5 OR l.inq_last_6mths IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
        END AS enq_pts \
        FROM itv009200_lending_club_new.loans_defaulters_detail_rec_enq_new l \
        INNER JOIN itv009200_lending_club_new.loans_defaulters_delinq_new d ON d.member_id = l.member_id  \
        INNER JOIN ph_pts p ON p.member_id = l.member_id where l.member_id NOT IN (select member_id from bad_data_customer)")

ldh_ph_df.show()

+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+
|           member_id|last_payment_pts|total_payment_pts|delinq_pts|public_records_pts|public_bankruptcies_pts|enq_pts|
+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+
|000c8875b71a6b47c...|             800|              650|       250|               800|                    800|    800|
|003769d7f54c7859e...|             500|              500|       250|               800|                    800|    800|
|003e1e6cbd2920bbb...|             500|              650|       250|               250|                    250|    800|
|004017b21bd4d6271...|             100|              650|       750|               800|                    800|    800|
|005b4c3db3fce07dc...|             500|              650|       250|               250|                    800|    250|
|00710707c563c2119...|             800| 

In [17]:
ldh_ph_df

member_id,last_payment_pts,total_payment_pts,delinq_pts,public_records_pts,public_bankruptcies_pts,enq_pts
000c8875b71a6b47c...,800,650,250,800,800,800
003769d7f54c7859e...,500,500,250,800,800,800
003e1e6cbd2920bbb...,500,650,250,250,250,800
004017b21bd4d6271...,100,650,750,800,800,800
005b4c3db3fce07dc...,500,650,250,250,800,250
00710707c563c2119...,800,650,250,800,800,800
007da79904f69970d...,800,650,250,800,800,800
00f435a80d0440ece...,500,500,100,800,800,800
00fc2ae3ffb1213e4...,650,650,250,800,800,800
00fc8144cb210ba8c...,500,650,250,250,250,800


In [18]:
ldh_ph_df.createOrReplaceTempView("ldh_ph_pts")

In [19]:
spark.sql("select * from ldh_ph_pts")

member_id,last_payment_pts,total_payment_pts,delinq_pts,public_records_pts,public_bankruptcies_pts,enq_pts
000c8875b71a6b47c...,800,650,250,800,800,800
003769d7f54c7859e...,500,500,250,800,800,800
003e1e6cbd2920bbb...,500,650,250,250,250,800
004017b21bd4d6271...,100,650,750,800,800,800
005b4c3db3fce07dc...,500,650,250,250,800,250
00710707c563c2119...,800,650,250,800,800,800
007da79904f69970d...,800,650,250,800,800,800
00f435a80d0440ece...,500,500,100,800,800,800
00fc2ae3ffb1213e4...,650,650,250,800,800,800
00fc8144cb210ba8c...,500,650,250,250,250,800


In [20]:
fh_ldh_ph_df = spark.sql("select ldef.*, \
   CASE \
   WHEN LOWER(l.loan_status) LIKE '%fully paid%' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%current%' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%in grace period%' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%late (16-30 days)%' OR LOWER(l.loan_status) LIKE '%late (31-120 days)%' THEN ${spark.sql.very_bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%charged off%' THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS loan_status_pts, \
   CASE \
   WHEN LOWER(a.home_ownership) LIKE '%own' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%rent' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%mortgage' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%any' OR LOWER(a.home_ownership) IS NULL THEN ${spark.sql.very_bad_rated_pts} \
   END AS home_pts, \
   CASE \
   WHEN l.funded_amount <= (a.total_high_credit_limit * 0.10) THEN ${spark.sql.excellent_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.10) AND l.funded_amount <= (a.total_high_credit_limit * 0.20) THEN ${spark.sql.very_good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.20) AND l.funded_amount <= (a.total_high_credit_limit * 0.30) THEN ${spark.sql.good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.30) AND l.funded_amount <= (a.total_high_credit_limit * 0.50) THEN ${spark.sql.bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.50) AND l.funded_amount <= (a.total_high_credit_limit * 0.70) THEN ${spark.sql.very_bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.70) THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS credit_limit_pts, \
   CASE \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A1' THEN ${spark.sql.excellent_rated_pts} \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A2' THEN (${spark.sql.excellent_rated_pts} * 0.95) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A3' THEN (${spark.sql.excellent_rated_pts} * 0.90) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A4' THEN (${spark.sql.excellent_rated_pts} * 0.85) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A5' THEN (${spark.sql.excellent_rated_pts} * 0.80) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B1' THEN (${spark.sql.very_good_rated_pts}) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B2' THEN (${spark.sql.very_good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B3' THEN (${spark.sql.very_good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B4' THEN (${spark.sql.very_good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B5' THEN (${spark.sql.very_good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C1' THEN (${spark.sql.good_rated_pts}) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C2' THEN (${spark.sql.good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C3' THEN (${spark.sql.good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C4' THEN (${spark.sql.good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C5' THEN (${spark.sql.good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D1' THEN (${spark.sql.bad_rated_pts}) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D2' THEN (${spark.sql.bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D3' THEN (${spark.sql.bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D4' THEN (${spark.sql.bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D5' THEN (${spark.sql.bad_rated_pts} * 0.80) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E1' THEN (${spark.sql.very_bad_rated_pts}) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E2' THEN (${spark.sql.very_bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E3' THEN (${spark.sql.very_bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E4' THEN (${spark.sql.very_bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E5' THEN (${spark.sql.very_bad_rated_pts} * 0.80) \
   WHEN (a.grade) in ('F', 'G') THEN (${spark.sql.unacceptable_rated_pts}) \
   END AS grade_pts \
   FROM ldh_ph_pts ldef \
   INNER JOIN itv009200_lending_club_new.loans l ON ldef.member_id = l.member_id \
   INNER JOIN itv009200_lending_club_new.customers_new a ON a.member_id = ldef.member_id where ldef.member_id NOT IN (select member_id from bad_data_customer)") 

In [21]:
fh_ldh_ph_df.createOrReplaceTempView("fh_ldh_ph_pts")

In [22]:
loan_score = spark.sql("SELECT member_id, \
((last_payment_pts+total_payment_pts)*0.20) as payment_history_pts, \
((delinq_pts + public_records_pts + public_bankruptcies_pts + enq_pts) * 0.45) as defaulters_history_pts, \
((loan_status_pts + home_pts + credit_limit_pts + grade_pts)*0.35) as financial_health_pts \
FROM fh_ldh_ph_pts")

In [23]:
loan_score.show()

+--------------------+-------------------+----------------------+--------------------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|
+--------------------+-------------------+----------------------+--------------------+
|000c8875b71a6b47c...|             290.00|               1192.50|            885.5000|
|003769d7f54c7859e...|             200.00|               1192.50|            402.5000|
|003e1e6cbd2920bbb...|             230.00|                697.50|            766.5000|
|004017b21bd4d6271...|             150.00|               1417.50|            822.5000|
|005b4c3db3fce07dc...|             230.00|                697.50|            619.5000|
|00710707c563c2119...|             290.00|               1192.50|            829.5000|
|007da79904f69970d...|             290.00|               1192.50|            822.5000|
|00f435a80d0440ece...|             200.00|               1125.00|            708.7500|
|00fc2ae3ffb1213e4...|             260.00| 

In [24]:
final_loan_score = loan_score.withColumn('loan_score', loan_score.payment_history_pts + loan_score.defaulters_history_pts + loan_score.financial_health_pts)

In [25]:
final_loan_score.createOrReplaceTempView("loan_score_eval")

In [26]:
loan_score_final = spark.sql("select ls.*, \
case \
WHEN loan_score > ${spark.sql.very_good_grade_pts} THEN 'A' \
WHEN loan_score <= ${spark.sql.very_good_grade_pts} AND loan_score > ${spark.sql.good_grade_pts} THEN 'B' \
WHEN loan_score <= ${spark.sql.good_grade_pts} AND loan_score > ${spark.sql.bad_grade_pts} THEN 'C' \
WHEN loan_score <= ${spark.sql.bad_grade_pts} AND loan_score  > ${spark.sql.very_bad_grade_pts} THEN 'D' \
WHEN loan_score <= ${spark.sql.very_bad_grade_pts} AND loan_score > ${spark.sql.unacceptable_grade_pts} THEN 'E'  \
WHEN loan_score <= ${spark.sql.unacceptable_grade_pts} THEN 'F' \
end as loan_final_grade \
from loan_score_eval ls")

In [27]:
loan_score_final.show()

+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|loan_score|loan_final_grade|
+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|000c8875b71a6b47c...|             290.00|               1192.50|            885.5000| 2368.0000|               B|
|003769d7f54c7859e...|             200.00|               1192.50|            402.5000| 1795.0000|               C|
|003e1e6cbd2920bbb...|             230.00|                697.50|            766.5000| 1694.0000|               C|
|004017b21bd4d6271...|             150.00|               1417.50|            822.5000| 2390.0000|               B|
|005b4c3db3fce07dc...|             230.00|                697.50|            619.5000| 1547.0000|               C|
|00710707c563c2119...|             290.00|               1192.50|            829

In [28]:
loan_score_final.createOrReplaceTempView("loan_final_table")

In [30]:
spark.sql("select * from loan_final_table where loan_final_grade in ('C')")

member_id,payment_history_pts,defaulters_history_pts,financial_health_pts,loan_score,loan_final_grade
003769d7f54c7859e...,200.00,1192.50,402.5000,1795.0000,C
003e1e6cbd2920bbb...,230.00,697.50,766.5000,1694.0000,C
005b4c3db3fce07dc...,230.00,697.50,619.5000,1547.0000,C
00fc8144cb210ba8c...,230.00,697.50,717.5000,1645.0000,C
017ce564dc0d6f975...,200.00,945.00,591.5000,1736.5000,C
017d1fd3d6169ee29...,260.00,697.50,694.7500,1652.2500,C
01d0c48835e969a01...,200.00,1192.50,262.5000,1655.0000,C
02f69de1384fcf78c...,150.00,945.00,864.5000,1959.5000,C
032503cc8f86dea72...,200.00,945.00,647.5000,1792.5000,C
035a8c46279041682...,230.00,1192.50,507.5000,1930.0000,C


In [24]:
spark.sql("select count(*) from loan_final_table")

count(1)
482842


In [25]:
loan_score_final.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv009200/lendingclubproject/processed/loan_score") \
.save()

In [ ]:
spark.stop()

In [26]:
!hadoop fs -ls /user/itv009200/lendingclubproject/processed/loan_score

Found 201 items
-rw-r--r--   3 itv009200 supergroup          0 2024-03-15 09:29 /user/itv009200/lendingclubproject/processed/loan_score/_SUCCESS
-rw-r--r--   3 itv009200 supergroup     169536 2024-03-15 09:28 /user/itv009200/lendingclubproject/processed/loan_score/part-00000-99be8d59-3c54-4368-b7fd-ad798e5eac8f-c000.snappy.parquet
-rw-r--r--   3 itv009200 supergroup     175834 2024-03-15 09:28 /user/itv009200/lendingclubproject/processed/loan_score/part-00001-99be8d59-3c54-4368-b7fd-ad798e5eac8f-c000.snappy.parquet
-rw-r--r--   3 itv009200 supergroup     171196 2024-03-15 09:28 /user/itv009200/lendingclubproject/processed/loan_score/part-00002-99be8d59-3c54-4368-b7fd-ad798e5eac8f-c000.snappy.parquet
-rw-r--r--   3 itv009200 supergroup     170342 2024-03-15 09:28 /user/itv009200/lendingclubproject/processed/loan_score/part-00003-99be8d59-3c54-4368-b7fd-ad798e5eac8f-c000.snappy.parquet
-rw-r--r--   3 itv009200 supergroup     166370 2024-03-15 09:28 /user/itv009200/lendingclubproject/proc

In [3]:
read = spark.read \
.format("parquet") \
.load("/user/itv009200/lendingclubproject/processed/loan_score")

In [4]:
read.show()

+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|loan_score|loan_final_grade|
+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|003f2e02514aea690...|             200.00|               1192.50|            560.8750| 1953.3750|               C|
|0054773d508bed06e...|             260.00|               1192.50|            381.5000| 1834.0000|               C|
|008054a22f213faad...|             230.00|                945.00|            469.0000| 1644.0000|               C|
|0081d6dbdc1913583...|             150.00|               1192.50|            780.5000| 2123.0000|               B|
|00eb5a401ffc73731...|             290.00|               1417.50|            444.5000| 2152.0000|               B|
|01156c673e503a06c...|             200.00|               1170.00|            787